In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

# Quick tour

¡Entra en marcha con los 🤗 Transformers! Comienza usando `pipeline()`para una inferencia veloz, carga un modelo preentrenado y un tokenizador con una [AutoClass](https://huggingface.co/docs/transformers/main/es/./model_doc/auto) para resolver tu tarea de texto, visión o audio.

<Tip>

Todos los ejemplos de código presentados en la documentación tienen un botón arriba a la izquierda para elegir entre Pytorch y TensorFlow.
Si no fuese así, se espera que el código funcione para ambos backends sin ningún cambio.

</Tip>

## Pipeline

`pipeline()`es la forma más fácil de usar un modelo preentrenado para una tarea dada.

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/tiZFewofSLM?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

El `pipeline()`soporta muchas tareas comunes listas para usar:

**Texto**:
* Análisis de Sentimientos: clasifica la polaridad de un texto dado.
* Generación de texto (solo en inglés): genera texto a partir de un input dado.
* Name entity recognition (NER): etiqueta cada palabra con la entidad que representa (persona, fecha, ubicación, etc.).
* Responder preguntas: extrae la respuesta del contexto dado un contexto y una pregunta.
* Fill-mask: rellena el espacio faltante dado un texto con palabras enmascaradas.
* Summarization: genera un resumen de una secuencia larga de texto o un documento.
* Traducción: traduce un texto a otro idioma.
* Extracción de características: crea una representación tensorial del texto.

**Imagen**:
* Clasificación de imágenes: clasifica una imagen.
* Segmentación de imágenes: clasifica cada pixel de una imagen.
* Detección de objetos: detecta objetos dentro de una imagen.

**Audio**:
* Clasificación de audios: asigna una etiqueta a un segmento de audio.
* Automatic speech recognition (ASR): transcribe datos de audio a un texto.

<Tip>

Para más detalles acerca del `pipeline()`y tareas asociadas, consulta la documentación [aquí](https://huggingface.co/docs/transformers/main/es/./main_classes/pipelines).

</Tip>

### Uso del Pipeline

En el siguiente ejemplo, usarás el `pipeline()`para análisis de sentimiento.

Instala las siguientes dependencias si aún no lo has hecho:

```bash
pip install tensorflow
```

Importa `pipeline()`y especifica la tarea que deseas completar:

In [ ]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")

El pipeline descarga y almacena en caché un [modelo preentrenado](https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english) por defecto y tokeniza para análisis de sentimiento. Ahora puedes usar `classifier` en tu texto objetivo:

In [ ]:
classifier("We are very happy to show you the 🤗 Transformers library.")

[{'label': 'POSITIVE', 'score': 0.9998}]

Para más de un enunciado entrega una lista de frases al `pipeline()`que devolverá una lista de diccionarios:

In [ ]:
results = classifier(["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."])
for result in results:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: POSITIVE, with score: 0.9998
label: NEGATIVE, with score: 0.5309

El `pipeline()`también puede iterar sobre un dataset entero. Comienza instalando la biblioteca [🤗 Datasets](https://huggingface.co/docs/datasets/):

```bash
pip install datasets
```

Crea un `pipeline()`con la tarea que deseas resolver y el modelo que quieres usar. Coloca el parámetro `device` a `0` para poner los tensores en un dispositivo CUDA:

In [ ]:
import torch
from transformers import pipeline

speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h", device=0)

A continuación, carga el dataset (ve 🤗 Datasets [Quick Start](https://huggingface.co/docs/datasets/quickstart.html) para más detalles) sobre el que quisieras iterar. Por ejemplo, vamos a cargar el dataset [SUPERB](https://huggingface.co/datasets/superb):

In [ ]:
import datasets

dataset = datasets.load_dataset("superb", name="asr", split="test")

Puedes pasar un pipeline para un dataset:

In [ ]:
files = dataset["file"]
speech_recognizer(files[:4])

[{'text': 'HE HOPED THERE WOULD BE STEW FOR DINNER TURNIPS AND CARROTS AND BRUISED POTATOES AND FAT MUTTON PIECES TO BE LADLED OUT IN THICK PEPPERED FLOWER FAT AND SAUCE'},
 {'text': 'STUFFERED INTO YOU HIS BELLY COUNSELLED HIM'},
 {'text': 'AFTER EARLY NIGHTFALL THE YELLOW LAMPS WOULD LIGHT UP HERE AND THERE THE SQUALID QUARTER OF THE BROTHELS'},
 {'text': 'HO BERTIE ANY GOOD IN YOUR MIND'}]

Para un dataset más grande, donde los inputs son de mayor tamaño (como en habla/audio o visión), querrás pasar un generador en lugar de una lista que carga todos los inputs en memoria. Ve la [documentación del pipeline](https://huggingface.co/docs/transformers/main/es/./main_classes/pipelines) para más información.

### Use otro modelo y otro tokenizador en el pipeline

El `pipeline()`puede adaptarse a cualquier modelo del [Model Hub](https://huggingface.co/models) haciendo más fácil adaptar el `pipeline()`para otros casos de uso. Por ejemplo, si quisieras un modelo capaz de manejar texto en francés, usa los tags en el Model Hub para filtrar entre los modelos apropiados. El resultado mejor filtrado devuelve un [modelo BERT](https://huggingface.co/nlptown/bert-base-multilingual-uncased-sentiment) multilingual fine-tuned para el análisis de sentimiento. Genial, ¡vamos a usar este modelo!

In [ ]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"

Usa `TFAutoModelForSequenceClassification`y ['AutoTokenizer'] para cargar un modelo preentrenado y un tokenizador asociado (más en un `TFAutoClass` debajo):

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Después puedes especificar el modelo y el tokenizador en el `pipeline()` y aplicar el `classifier` en tu texto objetivo:

In [ ]:
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier("Nous sommes très heureux de vous présenter la bibliothèque 🤗 Transformers.")

[{'label': '5 stars', 'score': 0.7273}]

Si no pudieras encontrar el modelo para tu caso respectivo de uso necesitarás ajustar un modelo preentrenado a tus datos. Mira nuestro [tutorial de fine-tuning](https://huggingface.co/docs/transformers/main/es/./training) para aprender cómo. Finalmente, después de que has ajustado tu modelo preentrenado, ¡por favor considera compartirlo (ve el tutorial [aquí](https://huggingface.co/docs/transformers/main/es/./model_sharing)) con la comunidad en el Model Hub para democratizar el NLP! 🤗

## AutoClass

In [ ]:
#@title
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/AhChOFRegn4?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

Debajo del capó, las clases `AutoModelForSequenceClassification`y `AutoTokenizer`trabajan juntas para dar poder al `pipeline()` Una [AutoClass](https://huggingface.co/docs/transformers/main/es/./model_doc/auto) es un atajo que automáticamente recupera la arquitectura de un modelo preentrenado con su nombre o el path. Sólo necesitarás seleccionar el `AutoClass` apropiado para tu tarea y tu tokenizador asociado con `AutoTokenizer`

Regresemos a nuestro ejemplo y veamos cómo puedes usar el `AutoClass` para reproducir los resultados del `pipeline()`

### AutoTokenizer

Un tokenizador es responsable de procesar el texto a un formato que sea entendible para el modelo. Primero, el tokenizador separará el texto en palabras llamadas *tokens*. Hay múltiples reglas que gobiernan el proceso de tokenización incluyendo el cómo separar una palabra y en qué nivel (aprende más sobre tokenización [aquí](https://huggingface.co/docs/transformers/main/es/./tokenizer_summary)). Lo más importante es recordar que necesitarás instanciar el tokenizador con el mismo nombre del modelo para asegurar que estás usando las mismas reglas de tokenización con las que el modelo fue preentrenado.

Carga un tokenizador con `AutoTokenizer`

In [ ]:
from transformers import AutoTokenizer

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Después, el tokenizador convierte los tokens a números para construir un tensor que servirá como input para el modelo. Esto es conocido como el *vocabulario* del modelo.

Pasa tu texto al tokenizador:

In [ ]:
encoding = tokenizer("We are very happy to show you the 🤗 Transformers library.")
print(encoding)

{'input_ids': [101, 11312, 10320, 12495, 19308, 10114, 11391, 10855, 10103, 100, 58263, 13299, 119, 102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

El tokenizador devolverá un diccionario conteniendo:

* [input_ids](https://huggingface.co/docs/transformers/main/es/./glossary#input-ids): representaciones numéricas de los tokens.
* [atttention_mask](https://huggingface.co/docs/transformers/main/es/.glossary#attention-mask): indica cuáles tokens deben ser atendidos.

Como con el `pipeline()` el tokenizador aceptará una lista de inputs. Además, el tokenizador también puede rellenar (pad, en inglés) y truncar el texto para devolver un lote (batch, en inglés) de longitud uniforme:

In [ ]:
tf_batch = tokenizer(
    ["We are very happy to show you the 🤗 Transformers library.", "We hope you don't hate it."],
    padding=True,
    truncation=True,
    max_length=512,
    return_tensors="tf",
)

Lee el tutorial de [preprocessing](https://huggingface.co/docs/transformers/main/es/./preprocessing) para más detalles acerca de la tokenización.

### AutoModel

🤗 Transformers provee una forma simple y unificada de cargar tus instancias preentrenadas. Esto significa que puedes cargar un `TFAutoModel`como cargarías un `AutoTokenizer` La única diferencia es seleccionar el `TFAutoModel`correcto para la tarea. Ya que estás clasificando texto, o secuencias, carga `TFAutoModelForSequenceClassification`

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

<Tip>
  Ve el [task summary](https://huggingface.co/docs/transformers/main/es/./task_summary) para revisar qué clase del `AutoModel`  deberías usar para cada tarea.
</Tip>

Ahora puedes pasar tu lote preprocesado de inputs directamente al modelo pasando las llaves del diccionario directamente a los tensores:

In [ ]:
tf_outputs = tf_model(tf_batch)

El modelo producirá las activaciones finales en el atributo `logits`. Aplica la función softmax a `logits` para obtener las probabilidades:

In [ ]:
import tensorflow as tf

tf_predictions = tf.nn.softmax(tf_outputs.logits, axis=-1)
print(tf.math.round(tf_predictions * 10**4) / 10**4)

tf.Tensor(
[[0.0021 0.0018 0.0116 0.2121 0.7725]
 [0.2084 0.1826 0.1969 0.1755  0.2365]], shape=(2, 5), dtype=float32)

<Tip>

Todos los modelos de 🤗 Transformers (PyTorch o TensorFlow) producirán los tensores *antes* de la función de activación
final (como softmax) porque la función de activación final es comúnmente fusionada con la pérdida.

</Tip>

Los modelos son [`torch.nn.Module`](https://pytorch.org/docs/stable/nn.html#torch.nn.Module) o [`tf.keras.Model`](https://www.tensorflow.org/api_docs/python/tf/keras/Model) estándares así que podrás usarlos en tu training loop usual. Sin embargo, para facilitar las cosas, 🤗 Transformers provee una clase `Trainer`para PyTorch que añade funcionalidades para entrenamiento distribuido, precición mixta, y más. Para TensorFlow, puedes usar el método `fit` desde [Keras](https://keras.io/). Consulta el [tutorial de entrenamiento](https://huggingface.co/docs/transformers/main/es/./training) para más detalles.

<Tip>

Los outputs del modelo de 🤗 Transformers son dataclasses especiales por lo que sus atributos pueden ser completados en un IDE.
Los outputs del modelo también se comportan como tuplas o diccionarios (e.g., puedes indexar con un entero, un slice o una cadena) en cuyo caso los atributos que son `None` son ignorados.

</Tip>

### Guarda un modelo

Una vez que tu modelo esté fine-tuned puedes guardarlo con tu tokenizador usando `TFPreTrainedModel.save_pretrained()`

In [ ]:
tf_save_directory = "./tf_save_pretrained"
tokenizer.save_pretrained(tf_save_directory)
tf_model.save_pretrained(tf_save_directory)

Cuando quieras usar el modelo otra vez cárgalo con `TFPreTrainedModel.from_pretrained()`

In [ ]:
tf_model = TFAutoModelForSequenceClassification.from_pretrained("./tf_save_pretrained")

Una característica particularmente cool de 🤗 Transformers es la habilidad de guardar el modelo y cargarlo como un modelo de PyTorch o TensorFlow. El parámetro `from_pt` o `from_tf` puede convertir el modelo de un framework al otro:

In [ ]:
from transformers import TFAutoModel

tokenizer = AutoTokenizer.from_pretrained(pt_save_directory)
tf_model = TFAutoModelForSequenceClassification.from_pretrained(pt_save_directory, from_pt=True)